In [3]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI
import pandas as pd
from dotenv import load_dotenv
import openai
import os



In [4]:
from dotenv import load_dotenv  
import os  
import openai
import os  
  

# Configure OpenAI API

AZURE_OPENAI_API_VERSION = "2023-03-15-preview"
AZURE_OPENAI_ENDPOINT ="https://cog-42as6n6i4ldsi.openai.azure.com/"
AZURE_OPENAI_API_KEY ="653103af9cd04492a2232b70327b7ce0"

# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"

In [5]:
print(os.environ["OPENAI_API_BASE"])
print(os.environ["OPENAI_API_KEY"])

https://cog-42as6n6i4ldsi.openai.azure.com/
653103af9cd04492a2232b70327b7ce0


In [32]:
import pandas as pd
import io
from azure.storage.blob import BlobServiceClient

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_test_embeddings10k.csv")

# Download the blob as a string
blob_string = blob_client.download_blob().content_as_text(encoding="utf-8")

# Load the string into a Pandas dataframe
df_test = pd.read_csv(io.StringIO(blob_string), encoding="utf-8")

# Print the dataframe
df_test.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Embeddings_complaint
0,2021-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,MY NAME IS XXXX XXXX AND I'M SUBMITTING THIS C...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,93727,NaN,Consent provided,Web,2021-09-17,Closed with non-monetary relief,Yes,NaN,4729991,"[-0.0461684875190258, -0.018764231353998184, 0..."
1,2020-05-26,Debt collection,Credit card debt,Took or threatened to take negative or legal a...,Collected or attempted to collect exempt funds,COLLECTION AGENCIES DEMAND PAYMENT ON ACCOUNTS...,NaN,Resurgent Capital Services L.P.,CA,90018,NaN,Consent provided,Web,2020-05-26,Closed with explanation,Yes,NaN,3668941,"[-0.02608240209519863, -0.027123533189296722, ..."
2,2022-05-01,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,NaN,"EQUIFAX, INC.",AZ,85041,NaN,Consent provided,Web,2022-05-01,Closed with explanation,Yes,NaN,5523897,"[-0.02903948351740837, -0.004012380726635456, ..."
3,2019-10-04,"Money transfer, virtual currency, or money ser...",Virtual currency,Fraud or scam,NaN,On Friday XX/XX/XXXX I searched in the web XXX...,NaN,JPMORGAN CHASE & CO.,TX,75043,NaN,Consent provided,Web,2019-10-04,Closed with explanation,Yes,NaN,3395771,"[-0.01851532608270645, 0.00250532990321517, 0...."
4,2022-01-09,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,"Dear Sir or Madam, I checked my personal credi...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NY,10305,NaN,Consent provided,Web,2022-01-09,Closed with explanation,Yes,NaN,5087511,"[-0.028908617794513702, -0.008078554645180702,..."


In [58]:
#count the distinct number of Products and print the name of the products
print(df_test['Sub-product'].unique())


['Credit reporting' 'Credit card debt' 'Virtual currency' 'Other mortgage'
 nan 'FHA mortgage' 'Non-federal student loan' 'I do not know'
 'Other banking product or service' 'Checking account' 'Other debt'
 'Other (i.e. phone, health club, etc.)'
 'General-purpose credit card or charge card' 'Mobile or digital wallet'
 'Savings account' 'Federal student loan servicing' 'Store credit card'
 'Conventional home mortgage' 'Credit repair services' 'Loan'
 'Personal line of credit' 'Payroll card'
 'Home equity loan or line of credit (HELOC)'
 'Other personal consumer report' 'VA mortgage' 'Medical debt'
 'Private student loan' 'Conventional fixed mortgage'
 'International money transfer' 'Private student loan debt'
 'General-purpose prepaid card' 'Vehicle loan' 'Gift card'
 'Payday loan debt' 'Vehicle lease' 'Payday loan' 'Installment loan'
 'Domestic (US) money transfer' 'Medical'
 'Home equity loan or line of credit' 'Credit card'
 'Conventional adjustable mortgage (ARM)' 'Auto debt'
 'Ref

In [35]:
from langchain.llms import OpenAI
from langchain.chat_models import AzureChatOpenAI
# To help construct our Chat Messages
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
llm=AzureChatOpenAI(deployment_name="gpt-35-turbo-16k", temperature=0.7)

# I like to use three double quotation marks for my prompts because it's easier to read
prompt = """
You are a financial expert. Give a short and very precise description and an example for a customer complaint for each of the following products in English:
{products}

The output has to be in the following JSON format:
[
    "Product": "Checking or savings account", "description": "A checking account is a deposit account held at a financial institution that allows withdrawals and deposits. Also called demand accounts or transactional accounts, checking accounts are very liquid and can be accessed using checks, automated teller machines, and electronic debits, among other methods.", "example": "example1"
    "Product": "Credit card or prepaid card", "description": "...", "example": "...",
    "Product": "Credit reporting, credit repair services, or other personal consumer reports", "description": "...", "example": "...",
    ...
]
Make sure it is a valid JSON string.
"""
#Extract the distint Products from df_test column Products
products = df_test['Product'].unique().tolist()
print(products)

#Parse the products to the prompt as the following string """-Product1\n-Product2\n-Product3\n...""
products = """-""" + """\n-""".join(products)
#print(products)

prompt = PromptTemplate(
    input_variables= ["products"],
    template=prompt
)
final_prompt = prompt.format(products=products)
print (f"Final Prompt: {final_prompt}")
print ("-----------")
output=llm([HumanMessage(content=final_prompt)])
print(output.content)










['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Money transfer, virtual currency, or money service', 'Mortgage', 'Credit card', 'Student loan', 'Credit reporting', 'Checking or savings account', 'Credit card or prepaid card', 'Payday loan', 'Vehicle loan or lease', 'Payday loan, title loan, or personal loan', 'Prepaid card', 'Bank account or service', 'Consumer Loan', 'Money transfers', 'Other financial service']
Final Prompt: 
You are a financial expert. Give a short and very precise description and an example for a customer complaint for each of the following products in English:
-Credit reporting, credit repair services, or other personal consumer reports
-Debt collection
-Money transfer, virtual currency, or money service
-Mortgage
-Credit card
-Student loan
-Credit reporting
-Checking or savings account
-Credit card or prepaid card
-Payday loan
-Vehicle loan or lease
-Payday loan, title loan, or personal loan
-Prepaid card
-Bank

[
    {"Product": "Credit reporting, credit repair services, or other personal consumer reports", "description": "These services provide individuals with information about their credit history, credit scores, and help to dispute any inaccuracies in their credit reports.", "example": "A customer complains that they have found incorrect information on their credit report and are unable to get it resolved."},
    {"Product": "Debt collection", "description": "Debt collection involves the process of pursuing payments of debts owed by individuals or businesses. Debt collectors may contact debtors via phone calls, letters, or other means to recover outstanding debts.", "example": "A customer complains that they are receiving aggressive and harassing phone calls from a debt collector."},
    {"Product": "Money transfer, virtual currency, or money service", "description": "These services enable individuals to transfer money electronically to another person or entity, often across different cou

In [36]:
print(output.content)

[
    {"Product": "Credit reporting, credit repair services, or other personal consumer reports", "description": "These services provide individuals with information about their credit history, credit scores, and help to dispute any inaccuracies in their credit reports.", "example": "A customer complains that they have found incorrect information on their credit report and are unable to get it resolved."},
    {"Product": "Debt collection", "description": "Debt collection involves the process of pursuing payments of debts owed by individuals or businesses. Debt collectors may contact debtors via phone calls, letters, or other means to recover outstanding debts.", "example": "A customer complains that they are receiving aggressive and harassing phone calls from a debt collector."},
    {"Product": "Money transfer, virtual currency, or money service", "description": "These services enable individuals to transfer money electronically to another person or entity, often across different cou

In [37]:
import json

# Parse the string into a list of dictionaries
examples = json.loads(output.content)

# Convert the list of dictionaries into the desired format
new_examples = []
for example in examples:
    new_example = {
        "Product": str(example["Product"]),
        "description": str(example["description"]),
        "example": str(example["example"])
    }
    new_examples.append(new_example)

In [38]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import AzureOpenAI

llm = OpenAI(engine="text-davinci-003",)


example_prompt = PromptTemplate(
    input_variables=["Product", "description", "example"],
    template="Product category: {Product}\nDescription: {description}\nCustomer complaint example: {example}\n",
)



WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [13]:
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}, 'engine': 'text-davinci-003'}


In [39]:
import openai
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://cog-42as6n6i4ldsi.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key ="653103af9cd04492a2232b70327b7ce0"



response = openai.Completion.create(
    engine="text-davinci-003",
    prompt="This is a test",
    max_tokens=5
)

In [ ]:
!pip install faiss-cpu

In [40]:
# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1), 
        
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS, 
    
    # This is the number of examples to produce.
    k=5
)

In [65]:
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="You are a financial expert and your task is to classify a customer complaint into a Product Category. You are given a customer complaint:\n{customer_complaint}\nYou have 5 examples of the Product categories, description and an example:",
    suffix="Classify the customer complaint into one Product Category. ONLY return the result of the Product Category:",
    
    # What inputs your prompt will receive
    input_variables=["customer_complaint"],
)

In [42]:
# Select a noun!
customer_complaint = "In accordance with the Fair Credit Reporting Act. The List of accounts below has violated my federally protected consumer rights to privacy and confidentiality under 15 USC 1681. Account Name : XXXX XXXX XXXX  Account # XXXX, Account Name : XXXX XXXX Account # XXXX has violated my rights. 15 U.S.C 1681 section 602 A. States I have the right to privacy. 15 U.S.C 1681 Section 604 A Section 2 : It also states a consumer reporting agency can not furnish an account without my written instructions 15 U.S.C 1681c. ( a ) ( 5 ) Section States : no consumer reporting agency may make any consumer report containing any of the following items of information or any other adverse item of information, other than records of convictions of crimes which antedates the report by more than seven years. 15 U.S.C. 1681s-2 ( A ) ( 1 ) A person shall not furnish any information relating to a consumer to any consumer reporting agency if the person knows or has reasonable cause to believe that the information is inaccurate."

# Generate the prompt
print(similar_prompt.format(customer_complaint=customer_complaint))

You are a financial expert and your task is to classify a customer complaint into a Product Category. You are given a customer complaint:
In accordance with the Fair Credit Reporting Act. The List of accounts below has violated my federally protected consumer rights to privacy and confidentiality under 15 USC 1681. Account Name : XXXX XXXX XXXX  Account # XXXX, Account Name : XXXX XXXX Account # XXXX has violated my rights. 15 U.S.C 1681 section 602 A. States I have the right to privacy. 15 U.S.C 1681 Section 604 A Section 2 : It also states a consumer reporting agency can not furnish an account without my written instructions 15 U.S.C 1681c. ( a ) ( 5 ) Section States : no consumer reporting agency may make any consumer report containing any of the following items of information or any other adverse item of information, other than records of convictions of crimes which antedates the report by more than seven years. 15 U.S.C. 1681s-2 ( A ) ( 1 ) A person shall not furnish any informati

In [51]:
print(len(df_test))

df_test_2=df_test.head(200)

10000


In [52]:
completion_response = llm(similar_prompt.format(customer_complaint=customer_complaint))
print(completion_response)



Credit reporting


In [66]:
# Use the llm function and feed in the column customer complaint from df

df_test_2["prediction"] = df_test_2["Consumer complaint narrative"].apply(lambda x: llm(similar_prompt.format(customer_complaint=x)))

#clean any new lines from the predictions
df_test_2["prediction"] = df_test_2["prediction"].apply(lambda x: x.replace("\n", ""))


/tmp/ipykernel_96751/3571149628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_2["prediction"] = df_test_2["Consumer complaint narrative"].apply(lambda x: llm(similar_prompt.format(customer_complaint=x)))
/tmp/ipykernel_96751/3571149628.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_2["prediction"] = df_test_2["prediction"].apply(lambda x: x.replace("\n", ""))


In [67]:
df_test_2.head()
#count df_test rows by prediction
len(df_test_2["prediction"].value_counts())
df_test_2.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Embeddings_complaint,prediction
0,2021-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,MY NAME IS XXXX XXXX AND I'M SUBMITTING THIS C...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,93727,NaN,Consent provided,Web,2021-09-17,Closed with non-monetary relief,Yes,NaN,4729991,"[-0.0461684875190258, -0.018764231353998184, 0...",Mortgage
1,2020-05-26,Debt collection,Credit card debt,Took or threatened to take negative or legal a...,Collected or attempted to collect exempt funds,COLLECTION AGENCIES DEMAND PAYMENT ON ACCOUNTS...,NaN,Resurgent Capital Services L.P.,CA,90018,NaN,Consent provided,Web,2020-05-26,Closed with explanation,Yes,NaN,3668941,"[-0.02608240209519863, -0.027123533189296722, ...",Debt collection
2,2022-05-01,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,NaN,"EQUIFAX, INC.",AZ,85041,NaN,Consent provided,Web,2022-05-01,Closed with explanation,Yes,NaN,5523897,"[-0.02903948351740837, -0.004012380726635456, ...",Credit reporting
3,2019-10-04,"Money transfer, virtual currency, or money ser...",Virtual currency,Fraud or scam,NaN,On Friday XX/XX/XXXX I searched in the web XXX...,NaN,JPMORGAN CHASE & CO.,TX,75043,NaN,Consent provided,Web,2019-10-04,Closed with explanation,Yes,NaN,3395771,"[-0.01851532608270645, 0.00250532990321517, 0....","Money transfer, virtual currency, or money ser..."
4,2022-01-09,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,"Dear Sir or Madam, I checked my personal credi...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NY,10305,NaN,Consent provided,Web,2022-01-09,Closed with explanation,Yes,NaN,5087511,"[-0.028908617794513702, -0.008078554645180702,...",Credit card


In [68]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  
  
# Assuming you have already loaded the dataframe 'df' with the relevant data  
  
# Extract the ground truth labels (actual products) and the predicted labels  
y_true = df_test_2['Product']  
y_pred = df_test_2['prediction']  
  
# Calculate evaluation metrics  
accuracy = accuracy_score(y_true, y_pred)  
precision = precision_score(y_true, y_pred, average='weighted')  
recall = recall_score(y_true, y_pred, average='weighted')  
f1 = f1_score(y_true, y_pred, average='weighted')  
  
# Print the evaluation results  
print("Accuracy:", accuracy)  
print("Precision:", precision)  
print("Recall:", recall)  
print("F1-score:", f1)  

Accuracy: 0.225
Precision: 0.7805974025974025
Recall: 0.225
F1-score: 0.223089743440637


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
from sklearn.metrics import classification_report  
  
# Assuming you have already loaded the dataframe 'df' with the relevant data  
  
# Extract the ground truth labels (actual products) and the predicted labels  
y_true = df_test_2['Product']  
y_pred = df_test_2['prediction']  
  
# Generate a classification report  
report = classification_report(y_true, y_pred)  
  
# Print the classification report  
print(report)   

                                                                              precision    recall  f1-score   support

                                                            Credit Reporting       0.00      0.00      0.00         0
                                                                 Credit card       0.00      0.00      0.00         0
                                                            Credit reporting       0.00      0.00      0.00         0
                          Money transfer, virtual currency, or money service       0.00      0.00      0.00         0
                                                                    Mortgage       0.00      0.00      0.00         0
                                                                 Payday Loan       0.00      0.00      0.00         0
                                                     Bank Account or Service       0.00      0.00      0.00         0
                                                     Ba

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
len(df_test_2)

20

In [70]:
# count df['prediction'] by row
df_test_2['prediction'].value_counts()

Credit reporting                                                                72
Bank account or service                                                         22
Debt collection                                                                 20
Credit Reporting                                                                17
Mortgage                                                                        13
Credit card                                                                     12
Student loan                                                                     7
Payday loan                                                                      6
Vehicle loan or lease                                                            4
Debt Collection                                                                  3
Bank Account or Service                                                          3
Credit card or prepaid card                                                      2
Mone